# Data Scraping Script for NLP Semantics Final Project

In this script, we are scraping the main text from the chosen news articles (listed in `valid_news_article_links.txt`) using the *`NewsPlease`* library. As we scrape the articles, we save the main text in a pandas DataFrame. This allows us to later analyze each data entry and extract sentences containing the keywords relevant to our final project. It is important to note that the articles are sourced from news outlets known to be as reliable as possible and minimally biased, as determined by [Ad Fontes Media’s Media Bias Chart](https://adfontesmedia.com/interactive-media-bias-chart/).

\* *[Documentation for NewPlease Library](https://github.com/fhamborg/news-please)*

In [ ]:
# libraries
from newsplease import NewsPlease
import pandas as pd
import time

# global variables
# the source domain and the corresponding news source
SOURCE_DOM_CONVERSION = {'www.nbcnews.com':'NBC News', 'www.npr.org':'NPR News', 
                         'www.voanews.com':'VOA News', 'www.upi.com':'UPI News',
                         'www.bbc.com':'BBC News', 'apnews.com':'AP News',}

# the path that contains all the links to the chosen news articles
FILE_PATH = 'valid_news_article_links.txt'

# the target words we are interested extracting sentences from in each article
TARGET_WORDS = ['Democrats', 'Democrat', 'Democratic', 'Republicans',
                'Republican', 'Liberals', 'Conservatives', ] 

## Scraping Main Text from News Articles

In [ ]:
# HELPER FUNCTIONS
def remove_duplicates(file_path):
    '''
    remove duplicates from our txt file that contains urls.
    '''

    with open(file_path, "r") as file:
        urls = file.readlines()

    # remove duplicates while preserving order
    unique_urls = list(dict.fromkeys(url.strip() for url in urls))

    # write back unique URLs to the file
    with open(file_path, "w") as file:
        file.write("\n".join(unique_urls))

    print(f'file updated. {len(urls) - len(unique_urls)} duplicates removed.')


In [ ]:
# dataframe where we will store the main text and source of the news articles
news_articles_df = pd.DataFrame(columns=['main text', 'source'])
data_entries = list()

# remove duplicates from the file
remove_duplicates(FILE_PATH)

with open(FILE_PATH, 'r') as file:
    for line in file:
        url = line.strip()
        if url:
            try: 
                article = NewsPlease.from_url(url)
                main_text = article.maintext
                source = SOURCE_DOM_CONVERSION.get(article.source_domain,
                                                   article.source_domain)
            except Exception as e:
                print(f"Error scrapping URL: {url} \nError: {e}")
                main_text = None
                source = None

            data_entries.append({'main text': main_text, 'source': source})
            # timeout to avoid being blocked 
            time.sleep(6)

# insert all the data entries into our dataframe
news_articles_df = pd.DataFrame(data_entries)

In [ ]:
print('shape of the dataframe:', news_articles_df.shape)
news_articles_df

,main text,source
0,The incoming Trump administration is consideri...,NBC News
1,President-elect Donald Trump has announced he'...,NPR News
2,President-elect Donald Trump is expected to co...,NPR News


## Data Cleaning/Selection Process